In [2]:
# With tf-idf

!pip install jsonlines
!pip install pandas
!pip install spacy
!pip install langdetect

import jsonlines
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from langdetect import detect

# Read the positions data from the CSV file
positions_df = pd.read_csv('/kaggle/input/positionscsv/positions.csv')

# Drop the row if the education is empty
positions_df = positions_df.dropna(subset=["education"])
positions_df = positions_df.reset_index(drop=True)

# Drop the row if the job description is not in English
positions_df['language'] = positions_df['description'].apply(detect)
positions_df = positions_df[positions_df['language'] == 'en']
positions_df = positions_df.drop(columns=['language'])

# Save the modified DataFrame to a CSV file
positions_df.to_csv('modified_positions.csv', index=False)

# Load the keywords from the JSON file
keywords = []
with jsonlines.open('/kaggle/input/jz-skill-patternsmaincore/jz_skill_patternsMain.jsonl') as json_file:
    for data in json_file:
        if data['label'] == 'COREPHD':
            patterns = data.get('pattern', [])
            for pattern in patterns:
                keyword = pattern.get('LOWER')
                if keyword:
                    keywords.append(keyword)

# Function to extract keywords from job description using TF-IDF with weight for duplications
def extract_keywords(job_description, keywords):
    vectorizer = TfidfVectorizer(vocabulary=keywords)
    doc = [job_description.lower()]
    vectorized_description = vectorizer.fit_transform(doc)
    feature_names = vectorizer.vocabulary_.keys()
    found_keywords = [keyword for keyword in feature_names if keyword in job_description.lower()]
    return found_keywords

# Create a list to store the job information
job_info = []

# Iterate over the positions
for _, row in positions_df.iterrows():
    job_id = row['_id']
    job_name = row['name']
    job_description = row['description']
    job_education = row['education']
    
    # Skip the job if the description is empty or missing
    if pd.isnull(job_description) or job_description == 'No job description found to import from MindMatch.':
        continue
    
    # Extract the keywords from the job description
    found_keywords = extract_keywords(job_description, keywords)
    
    # Calculate the score
    num_words = len(job_description.split())
    score = len(found_keywords)
    match = round(score / num_words * 100, 1)
    
    # Write the keywords to a text file
    with open(f'{job_id}_phds.txt', 'w') as txt_file:
        txt_file.write(f'Job Name: {job_name}\n')
        for keyword in found_keywords:
            txt_file.write(keyword + '\n')
    
    # Print the match percentage
    print(f"Job ID: {job_id}, Job_name: {job_name}, Match: {match}%, Num Words: {num_words}")

    # Add job information to the list
    job_info.append({'Job ID': job_id, 'Name': job_name, 'Education': job_education, 'Match': match, 'Num Words': num_words, 'Keywords': ', '.join(found_keywords)})

# Create a DataFrame from the job information
job_info_df = pd.DataFrame(job_info)

# Specify the path and filename for the Excel file
excel_file_path = '/kaggle/working/tf_idf.xlsx'

# Write the DataFrame to an Excel file
job_info_df.to_excel(excel_file_path, index=False)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 13.1 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993241 sha256=a09b4bc53245770d2995445032d17187bd649724f350c92b2e0dcf17b9c6661a
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Job ID: 8e7d7a00fb6701, Job_name: rest, Match: 1.0%, Num Words: 191
Job ID: 4b09219bb22d01, Job_name: Lead Scientist - Artificial Intelligence, Match: 0.6%, Num Words: 155
Job ID: 0564017952b801, Job_name: Mechatronics and Software Development Scientist H/F, Match: 0.5%, Num Words: 222
Job ID: c97c41c72de801, Job_name: Lead Software Developer H/F, Match: 0.9%, Num Words: 228
Job ID: 0286a158f05f01, Job_name: Researcher in Advanced Education, Match: 0.8%, Num Words: 247
Job ID: d2840269c20b01, Job_name: ANTIBODY ENGINEERING (SENIOR) SCIENTIST, Match: 0.6%, Num Words: 173
Job ID: a901ad66059901, Job_name: Biostatistician, Consulting Services (CDI) F/M, Match: 1.0%, Num Words: 289
Job ID: 7059e205499201, Job_name: Strategy and Management Consultant (CDI) F/M, Match: 1.1%, Num Words: 460
Job ID: 97a62c38f66801, Job_name: Strategy and Management Consultant (CDI) F/M, Match: 1.1%, Num Words: 460
Job ID: f0789d7c052501, Job_name: Advanced Analytics Specialist (CDI) F/M, Match: 0.7%, Num Words